In [1]:
import concurrent.futures
import csv
import json
import logging
import os
import random
import sys
from heapq import nlargest
import random
import openai
import torch
from tenacity import retry, stop_after_attempt, wait_exponential
from transformers import AutoModel, AutoTokenizer

TOP_K = 20
PART_NUM = "0"
i = 0
error_list=[]
openai.api_type = "azure"
openai.api_base = "https://biocodeeval-openai.openai.azure.com/"
openai.api_version = "2023-05-15"
openai.api_key = '' # get this API key from the resource (its not inside the OpenAI deployment portal)

key_bundles = [
    ('', "https://biocodeeval-openai.openai.azure.com/"),
    ('', "https://biocodeeval-openai2.openai.azure.com/"),
    ('', "https://biocodeeval-openai3.openai.azure.com/")
]

# Contriever part
tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
model = AutoModel.from_pretrained('facebook/contriever')

# Mean pooling
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings

def construct_prompt_for_user_profile(evidences,profile_type):
    prompt_str = ""
    if profile_type == "topics":
        prompt_str = "Given the following user's browsing history of science papers, Use a few words to describe the topics that could represent user's interests:\n"
        for index, evidence in enumerate(evidences):
            evidence_str = "History #"+str(index) +"\nContent: "+evidence["Content"] +"\nTitles： "+evidence["Title"]
        prompt_str = prompt_str + evidence_str +"\nTopics:"
    if profile_type == "keywords":
        prompt_str = "Given the following user's browsing history of science papers, extract the top common keywords in the titles that may represent users' interests:\n"	
        for index, evidence in enumerate(evidences):	
            evidence_str = "History #"+str(index) +"\nContent: "+evidence["Content"] +"\nTitles： "+evidence["Title"]	
        prompt_str = prompt_str + evidence_str +"\nKeywords:"
    return prompt_str

def run_openai(prompt,key_bundles,i):
    my_dict = {}
    my_dict["role"] = "user"
    my_dict["content"] = prompt
    l = []
    l.append(my_dict)
    result = ""
    try:
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
        result = response.choices[0].message["content"].replace('\n', ' ')
        print(indice+"\t"+result+"\n")
        return result
    except openai.error.Timeout:
        i = i+1
        print("Timeout",indice)
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        try:
            response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
            result = response.choices[0].message["content"].replace('\n', ' ')
            print(indice+"\t"+result+"\n")
            return result
        except openai.error.Timeout:
            i = i+1
            print("Timeout",indice)
            key_bundle = key_bundles[i%3]
            openai.api_key, openai.api_base = key_bundle
            try:
                response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
                result = response.choices[0].message["content"].replace('\n', ' ')
                print(indice+"\t"+result+"\n")
                return result
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                error_list.append(i)
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            error_list.append(i)
        #print(result)
    except openai.error.InvalidRequestError:
        print("InvalidRequestError",indice)
        error_list.append(i)
        #print(result)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        error_list.append(i)
    return result

with open('task_5_test_questions.json', 'r', encoding='utf-8') as f:
    jsonObject = json.load(f)
f.close()


profile_history = {}
input_dict = {}
profile_dict = {}
for item in jsonObject:
    input_dict[item["id"]] = item["input"]
    input_str = input_dict[item["id"]].replace("Generate a title for the following abstract of a paper: ","")
    profile_dict[item["id"]] = [input_str]
    profile_history[item["id"]] = [{"Content":"","Title":input_str}]
    for profile in item["profile"]:
        profile_history[item["id"]].append({"id":profile["id"],"Content":profile["abstract"],"Title":profile["title"]})
        profile_dict[item["id"]].append("Content: "+profile["abstract"]+"\nTitle: "+profile["title"])

with open("test_cot_prompt_input_history_user_profile_random.tsv", "wb", buffering=0) as out_file:
    for indice in profile_dict:
        top_score_docs = random.sample(profile_history[indice], min(TOP_K,len(profile_history[indice])))
        prompt = construct_prompt_for_user_profile(top_score_docs,"keywords")
        keywords = run_openai(prompt,key_bundles,i)
        i = i +1
        prompt = construct_prompt_for_user_profile(top_score_docs,"topics")
        topics = run_openai(prompt,key_bundles,i)
        i = i +1
        write_str = bytes(str(indice)+"\t"+keywords+"\t"+topics+"\n", 'utf-8')
        out_file.write(write_str)

        

420	strong truthfulness, externality-resistant mechanisms, auctions, multi-dimensional scheduling.

420	Mechanism design, auctions, scheduling, truthfulness, externalities.

421	identity-based public key cryptography, ID-PKC, traditional public key infrastructures, PKI, comparison, contrast, distinguishing features, benefits, drawbacks, technologies.

421	Public key cryptography, identity-based cryptography, PKI, ID-PKC, technology comparison.

422	BCI, P300 waves, EEG signals, 2-D motion, filter, GA, evolutionary.

422	Brain-computer interfaces, EEG signals, P300 waves, optimization algorithms, computer science, neuroscience.

423	power conservation, LTE, wireless networks, mobile devices, distributed multiagent system, scalability.

423	Wireless networks, power conservation, LTE, multiagent systems.

424	Cheating, Husbands, Stories, Case Study, Knowledge, Action, Communication, Distributed Environment.

424	Distributed environments, knowledge, communication, action, puzzles.

425	inf